In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from tfx.components import(
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Tuner
)
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


In [2]:
PIPELINE_NAME = 'grade-pipeline'
SCHEMA_PIPELINE_NAME = 'grade-tfdv-schema'

PIPELINE_ROOT = os.path.join('pipeline', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [3]:
grade = pd.read_csv("Data/data.csv")
grade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   StudyTimeWeekly  2392 non-null   int64
 1   Absences         2392 non-null   int64
 2   Tutoring         2392 non-null   int64
 3   ParentalSupport  2392 non-null   int64
 4   GPA              2392 non-null   int64
 5   GradeClass       2392 non-null   int64
dtypes: int64(6)
memory usage: 112.2 KB


In [4]:
'''plt.figure(figsize = (10,8))
correlation_matrix = grade.corr().round(2)

sns.heatmap(data = correlation_matrix, annot = True, cmap = 'rocket_r')'''

"plt.figure(figsize = (10,8))\ncorrelation_matrix = grade.corr().round(2)\n\nsns.heatmap(data = correlation_matrix, annot = True, cmap = 'rocket_r')"

In [5]:
'''data_path = "data"
grade = grade.astype('int64')
grade = grade.drop(["StudentID",
                    "Age",
                    "Gender",
                    "Ethnicity",
                    "ParentalEducation",
                    "Extracurricular",
                    "Sports",
                    "Music",
                    "Volunteering"], axis=1)
grade.to_csv(os.path.join(data_path, "data.csv"), index=False)'''

'data_path = "data"\ngrade = grade.astype(\'int64\')\ngrade = grade.drop(["StudentID",\n                    "Age",\n                    "Gender",\n                    "Ethnicity",\n                    "ParentalEducation",\n                    "Extracurricular",\n                    "Sports",\n                    "Music",\n                    "Volunteering"], axis=1)\ngrade.to_csv(os.path.join(data_path, "data.csv"), index=False)'

In [6]:
grade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   StudyTimeWeekly  2392 non-null   int64
 1   Absences         2392 non-null   int64
 2   Tutoring         2392 non-null   int64
 3   ParentalSupport  2392 non-null   int64
 4   GPA              2392 non-null   int64
 5   GradeClass       2392 non-null   int64
dtypes: int64(6)
memory usage: 112.2 KB


In [7]:
grade.head()

,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,GPA,GradeClass
0,19,7,1,2,2,2
1,15,0,0,1,3,1
2,4,26,0,2,0,4
3,10,14,0,3,2,3
4,4,17,1,3,1,4


In [8]:
DATA_ROOT = 'data'

In [9]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

In [10]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ]
    )
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [11]:
statistic_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(statistic_gen.outputs['statistics'])

In [13]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Absences',INT,required,,-
'GPA',INT,required,,-
'GradeClass',INT,required,,-
'ParentalSupport',INT,required,,-
'StudyTimeWeekly',INT,required,,-
'Tutoring',INT,required,,-


In [15]:
example_validator = ExampleValidator(
    statistics=statistic_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [16]:
interactive_context.show(example_validator.outputs['anomalies'])

In [17]:
TRANSFORM_MODULE_FILE = "transform.py"


In [18]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

NUMERICAL_FEATURES = [
    "StudyTimeWeekly",
    "Absences",
    "Tutoring",
    "ParentalSupport",
    "GPA"
]

LABEL_KEY = "GradeClass"

def transformed_name(key):
    return key + '_xf'

def preprocessing_fn(inputs):
    outputs = {}

    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(inputs[feature])
    
    # Transform the label
    outputs[transformed_name(LABEL_KEY)] = inputs[LABEL_KEY]
    
    return outputs

Overwriting transform.py


In [19]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipeline\grade-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\a5f77dd4750f45bb9841ddebcd04e712\assets


INFO:tensorflow:Assets written to: pipeline\grade-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\a5f77dd4750f45bb9841ddebcd04e712\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [38]:
TRAINER_MODULE_FILE = "trainer.py"

In [39]:
%%writefile trainer.py

import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.components.trainer.executor import TrainerFnArgs

LABEL_KEY = "GradeClass"
FEATURE_KEY = [
    "StudyTimeWeekly",
    "Absences",
    "Tutoring",
    "ParentalSupport",
    "GPA"
]

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=64, num_epochs=None) -> tf.data.Dataset:
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
        num_epochs=num_epochs
    )

    # Repeat the dataset indefinitely
    dataset = dataset.repeat()

    return dataset


def get_model():
    input_features = []
    for key in FEATURE_KEY:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(key))
        )

    concatenate = tf.keras.layers.concatenate(input_features)
    deep = tf.keras.layers.Dense(512, activation="relu")(concatenate)
    deep = tf.keras.layers.BatchNormalization()(deep)
    deep = tf.keras.layers.Dropout(0.2)(deep)

    deep = tf.keras.layers.Dense(256, activation="relu")(deep)
    deep = tf.keras.layers.BatchNormalization()(deep)
    deep = tf.keras.layers.Dropout(0.2)(deep)

    deep = tf.keras.layers.Dense(128, activation="relu")(deep)
    deep = tf.keras.layers.BatchNormalization()(deep)
    deep = tf.keras.layers.Dropout(0.2)(deep)

    outputs = tf.keras.layers.Dense(5, activation="softmax")(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0005),  # Adjust learning rate as needed
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, batch_size=64, num_epochs=1)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, batch_size=64, num_epochs=1)

    model = get_model()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback],
        epochs=10
    )

    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }

    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=signatures
    )

    plot_model(
        model,
        to_file="image/model_plot.png",
        show_shapes=True,
        show_layer_names=True
    )


Overwriting trainer.py


In [40]:
from tfx.proto import trainer_pb2
from tfx.components.trainer.component import Trainer

trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=200), 
)

interactive_context.run(trainer)


Epoch 1/10
5000/5000 [==============================] - 63s 12ms/step - loss: 0.4665 - accuracy: 0.8450 - val_loss: 0.6377 - val_accuracy: 0.8176
Epoch 2/10
5000/5000 [==============================] - 59s 12ms/step - loss: 0.3541 - accuracy: 0.8640 - val_loss: 0.7488 - val_accuracy: 0.7945
Epoch 3/10
5000/5000 [==============================] - 59s 12ms/step - loss: 0.2929 - accuracy: 0.8797 - val_loss: 0.9150 - val_accuracy: 0.7694
Epoch 4/10
5000/5000 [==============================] - 59s 12ms/step - loss: 0.2523 - accuracy: 0.8927 - val_loss: 1.1075 - val_accuracy: 0.7442
Epoch 5/10
5000/5000 [==============================] - 58s 12ms/step - loss: 0.2231 - accuracy: 0.9038 - val_loss: 1.2018 - val_accuracy: 0.7379
Epoch 6/10
5000/5000 [==============================] - 59s 12ms/step - loss: 0.2022 - accuracy: 0.9120 - val_loss: 1.2891 - val_accuracy: 0.7484
Epoch 7/10
2319/5000 [============>.................] - ETA: 37s - loss: 0.1885 - accuracy: 0.9181

KeyboardInterrupt: 